In [1]:
from ift6758.client.serving_client import ServingClient
from ift6758.client.game_client import GameClient

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
Game_client = GameClient()
Serving_client = ServingClient()
workspace = "ift-6758-projet-quipe-13"
model = 'xgboost-base-all-features'
version = '1.0.0'

print(Game_client.gameId)
print(Game_client.game_ended)
# game_id = 2021020329 # int or str ??
# game_id = 2021020336
game_id = 2022020504

df = Game_client.process_query(game_id, model_name=model) # model_name="..."
df

teams = df['team'].unique()
print((teams[0]))
print((teams[1]))


0
None
FLA
BOS


In [5]:
out = Serving_client.download_registry_model(workspace, model, version)
pred_MODEL = Serving_client.predict(df)

# df = pd.DataFrame(df, columns=Serving_client.features) 
# df['Model Output'] = pred_MODEL

df
pred_MODEL

,0
0,0.040112
1,0.009077
2,0.099093
3,0.092904
4,0.007955
5,0.092184
6,0.095405
7,0.008094
8,0.013158
9,0.023936


In [6]:
def calculate_game_goals(df: pd.DataFrame, pred):
    """
    Sum over model_pred for each team 
        Input: df (DataFrame), with feature values and model prediction for every event
        Output: pred_goals (list), predicted number of goals for each team    
    """
    # pred_goals = [1.8, 3.4] 
    df = df.reset_index(drop=True)
    df['Model Output'] = pred

    teams = df['team'].unique()

    pred_team1 = df.loc[df['team']==teams[0] , 'Model Output']
    sum_pred_team1 = pred_team1.sum()
    pred_team2 = df.loc[df['team']==teams[1] , 'Model Output']
    sum_pred_team2 = pred_team2.sum()

    real_team1 = df.loc[df['team']==teams[0] , 'isGoal']
    sum_real_team1 =real_team1.sum()
    real_team2 = df.loc[df['team']==teams[1] , 'isGoal']
    sum_real_team2 = real_team2.sum()
 
    pred_goals = [sum_pred_team1, sum_pred_team2]
    real_goals = [sum_real_team1, sum_real_team2]

    return pred_goals, real_goals, teams, df


In [7]:
pred_goals, real_goals, teams, df1 = calculate_game_goals(df, pred_MODEL)

print(pred_goals)
print(real_goals)
df1

[2.5849812920205295, 2.1939832223579288]
[2, 7]


,team,isGoal,periodTimeSec,period,coordinate_x,coordinate_y,distanceFromGoal,shotAngle,lastEventCoord_x,lastEventCoord_y,...,BLOCKED_SHOT,FACEOFF,GIVEAWAY,GOAL,HIT,MISSED_SHOT,PENALTY,SHOT,TAKEAWAY,Model Output
0,FLA,0,32.0,1,-35.0,15.0,56.044625,-0.270947,-69.0,22.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040112
1,FLA,0,198.0,1,-48.0,40.0,57.280014,-0.773053,20.0,22.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009077
2,BOS,0,325.0,1,64.0,6.0,25.709920,0.235545,-31.0,36.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.099093
3,BOS,0,522.0,1,82.0,-7.0,9.899495,-0.785398,69.0,-22.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092904
4,FLA,0,640.0,1,-74.0,-34.0,37.161808,1.155304,-73.0,-34.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.007955
5,BOS,1,651.0,1,78.0,11.0,15.556349,0.785398,-74.0,-34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092184
6,BOS,0,681.0,1,82.0,-11.0,13.038405,-1.004067,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095405
7,FLA,0,744.0,1,-19.0,-14.0,71.386273,0.197396,-7.0,35.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008094
8,FLA,0,805.0,1,82.0,24.0,172.675997,-0.139440,59.0,33.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.013158
9,BOS,0,821.0,1,42.0,23.0,52.325902,0.455101,-37.0,-32.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023936


In [8]:
GOALS = Game_client.get_real_goals()
print(GOALS)

[3, 7]


In [5]:
df['pred'] = pred_MODEL
df1 = df.reset_index()
df2 = df.reset_index(drop=True)

In [6]:
df2['pred2'] = pred_MODEL

In [5]:
pred_team1 = df.loc[df['team']==teams[0] , 'Model Output']
sum_pred_team1 = pred_team1.sum()
pred_team2 = df.loc[df['team']==teams[1] , 'Model Output']
sum_pred_team2 = pred_team2.sum()
print(pred_team1)

real_team1 = df.loc[df['team']==teams[0] , 'isGoal']
sum_real_team1 =real_team1.sum()
print(real_team1)


pred_goals = [sum_pred_team1, sum_pred_team2]

new_pred_goals = [0, 0]
new_pred_goals[0] += pred_goals[0]
new_pred_goals[1] += pred_goals[1]
new_pred_goals[0] += pred_goals[0]
new_pred_goals[1] += pred_goals[1]

for i in range(len(pred_goals)):
    new_pred_goals[i] += pred_goals[i]
   


print(sum_pred_team1)
print(sum_real_team1)

print(pred_goals)
print(new_pred_goals)

KeyError: 'team'

In [32]:
import datetime
# last_period = df.loc[, 'period']
# print(last_period)
last_period = df['period'].values[-1:]
last_period_time = df['periodTimeSec'].values[-1:]

print(int(last_period))
print(int(last_period_time))
print(type(last_period_time))

last_period_time 
# p_time = pd.to_datetime(last_period_time, format="%S")
p_time = str(datetime.timedelta(seconds=int(last_period_time), hours = 0))
print(p_time)

import time

p_time2 = time.strftime("%M:%S", time.gmtime(int(last_period_time)))
print(type(p_time2))
print(p_time2)

# df["periodTimeSec"] = (pd.to_datetime(df["periodTimeSec"], format="%M:%S") - pd.to_datetime("0:0", format="%M:%S")).dt.total_seconds()

3
1060
<class 'numpy.ndarray'>
0:17:40
<class 'str'>
17:40


In [9]:
import os
os.environ.get("COMET_API_KEY")

'j3MtAk6zQdcxIE1jZWiq4HTrV'

In [33]:
features = ['Feature1', 'Feature2','Feature3','Feature4','Feature5',
            'Feature6', 'Feature7', 'Feature8', 'Feature9', 'Feature10',
            'Feature11', 'Feature12', 'Feature13', 'Feature14', 'Feature15',
            'Feature16', 'Feature17', 'Feature18']
feature_values = np.random.rand(5, 18) 
model_pred = np.random.rand(5,1)

df2 = pd.DataFrame(feature_values, columns=features)
df2['Model Output'] = model_pred


df2 = pd.concat([df2, None])

df2

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Model Output
0,0.565897,0.437001,0.166990,0.645237,0.873760,0.352968,0.114294,0.594211,0.362123,0.272912,0.308872,0.880309,0.771296,0.942122,0.013075,0.045918,0.474948,0.878328,0.099758
1,0.982096,0.492012,0.862581,0.577682,0.261515,0.319963,0.733667,0.792846,0.788939,0.637075,0.422281,0.210838,0.695833,0.954152,0.186321,0.561371,0.129717,0.477895,0.996569
2,0.041166,0.065437,0.511848,0.883133,0.709260,0.762317,0.658555,0.985552,0.739159,0.358716,0.972950,0.223840,0.514194,0.718356,0.858904,0.179396,0.303923,0.100735,0.627963
3,0.754381,0.572775,0.176649,0.172851,0.197924,0.031713,0.658451,0.470891,0.281447,0.974577,0.986440,0.486588,0.117464,0.954590,0.790059,0.571221,0.923999,0.867535,0.259576
4,0.415895,0.223875,0.039468,0.081874,0.948728,0.157731,0.782977,0.793000,0.735247,0.879051,0.093437,0.928591,0.682784,0.733599,0.281980,0.752939,0.059764,0.263481,0.524554


In [39]:
pred_team1 = df2.loc[df2['Feature1']==10 , 'Model Output']
print(pred_team1)
sum_pred_team1 = pred_team1.sum()
print(sum_pred_team1)

Series([], Name: Model Output, dtype: float64)
0.0


In [4]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [5]:
hey.text

'Hello!'

In [1]:
# default_model = {
#     'workspace': 'ift-6758-projet-quipe-13',
#     'model': 'xgboost-best-all-features',
#     'version': '1.0.0'
# }
# model = requests.post("http://127.0.0.1:8080/download_registry_model", json=default_model)
# model.text

In [2]:
# %xmode Verbose
# %tb

In [40]:
from ift6758.data.fetch_data import FetchData

fetch = FetchData()
data = fetch.get_play_by_play(2017020235)

data


{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'gamePk': 2017020235,
 'link': '/api/v1/game/2017020235/feed/live',
 'metaData': {'wait': 10, 'timeStamp': '20171113_082730'},
 'gameData': {'game': {'pk': 2017020235, 'season': '20172018', 'type': 'R'},
  'datetime': {'dateTime': '2017-11-10T00:00:00Z',
   'endDateTime': '2017-11-10T02:47:39Z'},
  'status': {'abstractGameState': 'Final',
   'codedGameState': '7',
   'detailedState': 'Final',
   'statusCode': '7',
   'startTimeTBD': False},
  'teams': {'away': {'id': 22,
    'name': 'Edmonton Oilers',
    'link': '/api/v1/teams/22',
    'venue': {'id': 5100,
     'name': 'Rogers Place',
     'link': '/api/v1/venues/5100',
     'city': 'Edmonton',
     'timeZone': {'id': 'America/Edmonton', 'offset': -7, 'tz': 'MST'}},
    'abbreviation': 'EDM',
    'triCode': 'EDM',
    'teamName': 'Oilers',


In [42]:
# x = data["liveData"].get("linescore")
# print(x)
# print(type(x))

goal1 = None
goal2 = None

if data["liveData"].get("linescore") is not None:
    goal1 = data["liveData"]["linescore"]["teams"]["home"]["goals"]
    goal2 = data["liveData"]["linescore"]["teams"]["away"]["goals"]

if goal1 is None and goal2 is None: # Bug while trying to get goal num from linescore
    print('hello')
    goal1 = data["liveData"]["plays"]["currentPlay"]["about"]["goals"]["home"]
    goal2 = data["liveData"]["plays"]["currentPlay"]["about"]["goals"]["away"]

print(goal1, goal2)
print(type(goal1))
# print(goal1)

away = data["gameData"]["teams"]["away"]["triCode"]
home = data["gameData"]["teams"]["home"]["triCode"]

print(away, home)

period = data["liveData"]["plays"]["currentPlay"]["about"]["period"]
periodTimeRemaining = data["liveData"]["plays"]["currentPlay"]["about"]["periodTimeRemaining"] 
print(period, periodTimeRemaining)

2 3
<class 'int'>
EDM NJD
4 00:17
